In [ ]:
import os
import gradio as gr
from langchain_groq import ChatGroq
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.prompts import ChatPromptTemplate
from langchain.prompts import MessagesPlaceholder
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough
from langchain_core.messages import trim_messages, HumanMessage

# Load environment variables
from dotenv import load_dotenv
load_dotenv()

# Load the model
llm = ChatGroq(model="llama-3.1-8b-instant",api_key='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')

# Track chat history
store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

# Define the prompt template
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a knowledgeable and authoritative guide on organ donation and medical topics. "
               "Your role is to provide accurate, reliable guidance on organ donation procedures, "
               "medical conditions, and related healthcare matters. You must strictly refuse to answer any "
               "question that is not related to organ donation or medical topics. If a question is unrelated, "
               "respond with: 'This question is outside the scope of Donix. I cannot answer it.' "
               "Remember past conversations to provide context-aware responses in {language}."),
    MessagesPlaceholder(variable_name="messages")
])

# Manage chat history
trimmer = trim_messages(
    max_tokens=900,
    strategy="last",
    token_counter=llm,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

# Create a chain
chain = (
    RunnablePassthrough.assign(messages=itemgetter("messages") | trimmer)
    | prompt
    | llm
)

# Set up session configuration
config = {"configurable": {"session_id": "default_session"}}

# Integrate chat history
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)

# Define Gradio interface
def chat_interface(message, language="English", session_id="default_session"):
    response = with_message_history.invoke(
        {
            "messages": HumanMessage(message),
            "language": language
        },
        config={"configurable": {"session_id": session_id}}
    ).content
    return response

iface = gr.Interface(
    fn=chat_interface,
    inputs=[
        gr.Textbox(label="Message"),
        gr.Dropdown(["English", "Hindi", "Sanskrit"], label="Language", value="English"),
        gr.Textbox(label="Session ID", value="default_session")
    ],
    outputs="text",
    title="Donix Organ Donation Chatbot",
    description="A chatbot specialized in organ donation and medical guidance."
)

if __name__ == "__main__":
    iface.launch()